In [1]:
import os
import sys
import asyncio
import joblib
import json
import threading
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import alpaca_trade_api as api
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from dotenv import load_dotenv
from flask_cors import CORS
from concurrent.futures import ThreadPoolExecutor
from alpaca_trade_api.rest import REST
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify, send_from_directory
from LSTM import restAPI, ALPACA_CONFIG, ML_Trend, Alpaca
from alpaca.trading.stream import TradingStream
from pydantic import BaseModel, Field
from typing import Optional,List

2025-03-29 09:41:24 | lumibot.credentials | WARNING | No .env file found. This is ok if you are using environment variables or secrets (like on Replit, AWS, etc), but if you are not, please create a .env file in the root directory of the project.


In [2]:
load_dotenv("setup.env")

ALPACA_CONFIG = {
    'API_KEY': os.getenv("APCA_API_KEY_ID"),
    'API_SECRET': os.getenv("APCA_API_SECRET_KEY"),
    "BASE_URL": "https://paper-api.alpaca.markets"  
}
restAPI = REST(ALPACA_CONFIG['API_KEY'], ALPACA_CONFIG['API_SECRET'], ALPACA_CONFIG['BASE_URL'])

try:
    account = restAPI.get_account()
except Exception as e:
    print(f"Authentication failed: {e}")

broker = Alpaca(ALPACA_CONFIG)
trading_ws = None
client = StockHistoricalDataClient(ALPACA_CONFIG['API_KEY'],ALPACA_CONFIG['API_SECRET'])

2025-03-29 09:43:46 | asyncio | INFO | [unknown] Waiting for the socket stream connection to be established, 
                method _stream_established must be called


Paper mode: False, Endpoint selected: BaseURL.TRADING_STREAM_PAPER


2025-03-29 09:43:46 | alpaca.trading.stream | INFO | started trading stream
2025-03-29 09:43:46 | alpaca.trading.stream | INFO | starting trading websocket connection


Attempting to connect to WebSocket...
WebSocket connected successfully to BaseURL.TRADING_STREAM_PAPER
WebSocket connected. Attempting authentication...
Attempting authentication with key: PKYA2RUW3CFL01GRDM6G
Sending authentication request: {"action": "auth", "key": "PKYA2RUW3CFL01GRDM6G", "secret": "Ts8JQ6KKdn4S34WT01ovPiIhtnGAssokMfzVvY42"}


2025-03-29 09:43:48 | alpaca.trading.stream | INFO | Connected to: BaseURL.TRADING_STREAM_PAPER


Received authentication response: b'{"stream":"authorization","data":{"action":"authenticate","status":"authorized"}}'
Authentication successful!
Subscribed to trade updates.


In [6]:
def get_historical_prices(symbol, days=1000, timeframe='day'):
    """
    Fetch historical price data for a stock symbol from Alpaca.

    Parameters:
        symbol (str): The stock symbol.
        days (int): Number of days to retrieve historical data for.
        timeframe (str): The timeframe for the price data (e.g., 'day', 'minute').

    Returns:
        DataFrame: A pandas DataFrame with historical price data.
    """
    # Define the start and end date for the data
    end_date = datetime.today()
    start_date = end_date - timedelta(days=days)

    timeframe_map = {
        'minute': TimeFrame.Minute,
        'hour': TimeFrame.Hour,
        'day': TimeFrame.Day,
        'week': TimeFrame.Week,
        'month': TimeFrame.Month
    }
    
    if timeframe not in timeframe_map:
        raise ValueError(f"Invalid timeframe: {timeframe}. Use one of {list(timeframe_map.keys())}")

    alpaca_timeframe = timeframe_map[timeframe]

    request_params = StockBarsRequest(
        symbol_or_symbols=[symbol],
        timeframe=alpaca_timeframe,
        start=start_date.strftime('%Y-%m-%d'),
        end=end_date.strftime('%Y-%m-%d'),
        feed="iex"
    )

    # Fetch historical price data
    barset = client.get_stock_bars(request_params)
    # barset = api.get_barset(symbol, timeframe, start=start_date, end=end_date)
    # Convert the data to a pandas DataFrame
    if symbol in barset:
        data = barset[symbol]
        data_dict = {
            'time': [bar.timestamp for bar in data],
            'open': [bar.open for bar in data],
            'high': [bar.high for bar in data],
            'low': [bar.low for bar in data],
            'close': [bar.close for bar in data],
            'volume': [bar.volume for bar in data]
        }
        df = pd.DataFrame(data_dict)
        df.set_index('time', inplace=True)
        # If you want to remove rows with missing values, you can do:
        df.dropna(subset=['close'], inplace=True)
        print("DF",df)
        return df
    else:
        print("BARSET",barset)
        return pd.DataFrame(barset)

In [9]:
data = get_historical_prices("GOOG")

BARSET data={'GOOG': [{   'close': 2279.07,
    'high': 2279.07,
    'low': 2129.1,
    'open': 2150.31,
    'symbol': 'GOOG',
    'timestamp': datetime.datetime(2022, 7, 5, 4, 0, tzinfo=TzInfo(UTC)),
    'trade_count': 2597.0,
    'volume': 74715.0,
    'vwap': 2234.008528}, {   'close': 2302.75,
    'high': 2326.5,
    'low': 2250.1,
    'open': 2282.69,
    'symbol': 'GOOG',
    'timestamp': datetime.datetime(2022, 7, 6, 4, 0, tzinfo=TzInfo(UTC)),
    'trade_count': 2383.0,
    'volume': 64627.0,
    'vwap': 2292.387852}, {   'close': 2386.22,
    'high': 2395.345,
    'low': 2313.7,
    'open': 2319.875,
    'symbol': 'GOOG',
    'timestamp': datetime.datetime(2022, 7, 7, 4, 0, tzinfo=TzInfo(UTC)),
    'trade_count': 2495.0,
    'volume': 60909.0,
    'vwap': 2362.738331}, {   'close': 2403.47,
    'high': 2406.4,
    'low': 2352.93,
    'open': 2352.93,
    'symbol': 'GOOG',
    'timestamp': datetime.datetime(2022, 7, 8, 4, 0, tzinfo=TzInfo(UTC)),
    'trade_count': 2135.0,
    'v

In [1]:
data.to_json()

NameError: name 'data' is not defined